In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

from calculator import *

rank_methods = RankMethods()

data = pd.read_csv('../data/res_evaluated.csv')
pd.set_option('display.max_columns',200)

3.1(Кореляційний аналіз):
Взяти дві змінні для дослідження кореляційної залежності.<br>
Для теста №1 за одну змінну взяти  дані деякої групи, в якості другої змінної взяти оцінки з математики <br>
Для теста №2 за одну змінну взяти бали мотивації для математичного предмета, в якості другої змінної бали мотивації цих же студентів для гуманітарного предмета.<br>
Для теста №3 за одну змінну взяти бали мотивації за шкалою “професійна діяльність”, в якості другої змінної бали мотивації цих же студентів для якоїсь іншої шкали.<br>

In [2]:
test_1 = {'x': data.loc[data['Курс']==4]['Улюблений предмет'].values, 'y':data.loc[data['Курс']==4]['Оцінка з цього предмету:'].values}
test_2 = {'x': data.loc[data['Курс']==4]['Test#2: Рівняння мат. фізики'].values, 'y':data.loc[data['Курс']==4]['Test#2: Технології прикладного програмування'].values}
test_3 = {'x': data[data['Курс']==4]['Професійні мотиви'].values, 'y': data[data['Курс']==4]['Мотиви творчої самореалізації'].values}

*	Знайти вибіркові коефіцієнту кореляції Пірсона, Спірмена та Кенделла та перевірити їхню значущість за допомогою параметричного критерію Пірсона та непараметричних Спірмена та Кенделла.
*	Застосувати можливості Пайтона для знаходження коефіцієнту кореляції Пірсона, Спірмена та Кенделла і перевірки цих коефіцієнтів на значущість. 

In [3]:
def t_value(r,x):
    """
    t_value with n-2 ddof under the null hypothesis
    """
    n = x.shape[0]
    return r * np.sqrt(n-2) / np.sqrt(1-r**2)
def p_value_rho(r,x):
    """
    corresponding p_value
    """
    n = x.shape[0]
    t = t_value(r,x)
    return stats.t.sf(t,df=n-2)#1-t.cdf(t,n-2)

In [4]:
def pearson_correlation(x,y):
    """
    Pearson correlation coefficient r and p-value p for testing non-correlation
    """
    def cov(x,y):
        res = (x - mean(x))*(y-mean(y))
        return res.sum()/(x.shape[0]-1)
    
    def corr(x,y):
        return cov(x,y)/(stddev(x)*stddev(y))
    
    r = corr(x,y)
    p = 2*(p_value_rho(r,x))
    return r,p

In [5]:
pearson_correlation(test_1['x'],test_1['y'])

(0.5115483144972256, 0.004563363106501617)

In [6]:
stats.pearsonr(test_1['x'],test_1['y'])

(0.5115483144972258, 0.004563363106501603)

In [7]:
def spearman_correlation(x,y):
    """
    Spearman correlation coefficient r and p-value p for testing non-correlation
    """
    def corr(x,y):
        rank_x,rank_y = rank_methods.rank(x), rank_methods.rank(y)
        if rank_methods.areDistinct(x,y):
            ranks_sum,n = rank_methods.rank_difference_sum(rank_x,rank_y),x.shape[0]
            return 1 - (6*ranks_sum)/(n**3-n)
        return pearson_correlation(rank_x,rank_y)[0]

    r = corr(x,y)
    p = 2*(p_value_rho(r,x))
    return r,p

In [8]:
spearman_correlation(test_1['x'],test_1['y'])

(0.5412714348395539, 0.0024282474368559678)

In [9]:
stats.spearmanr(test_1['x'],test_1['y'])

SpearmanrResult(correlation=0.5412714348395541, pvalue=0.0024282474368559556)

In [10]:
def z_value_tau(r,x):
    t,n = t_value(r,x), x.shape[0]
    return 3 * t * np.sqrt(n**2-n) / np.sqrt(4*n+10)

In [14]:
def kendall_correlation(x,y):
    """
    Kendall correlation coefficient r and p-value p for testing non-correlation
    """
    def corr(x,y):
        n = x.shape[0]
        concordant,discordant = concordant_discordant(y)
        c,d,t,u = concordant_discordant_ties(x,y)
        return (c-d)/np.sqrt((c+d+t)*(c+d+u))
    
    r = corr(x,y)
    p = stats.norm.cdf(z_value_tau(r,x))
    return r,p

In [15]:
kendall_correlation(test_1['x'],test_1['y'])

(0.4586562835474408, 1.0)

In [16]:
stats.kendalltau(test_1['x'],test_1['y'])

KendalltauResult(correlation=0.42971314287974627, pvalue=0.0030388458780621253)

3.2(Регресійний аналіз):

*	Побудувати регресійну модель залежності однієї змінної  від іншої методом найменших квадратів та за умови відомого коефіцієнта кореляції Пірсона.
*	Побудувати регресійну модель в Пайтоні,  провести аналіз залишків різними методами. 

In [ ]:
from sklearn.base import BaseEstimator

class OLS(BaseEstimator):
    def __init__(self):
        super(OLS,self).__init__()
        
    def fit(self,X,y=None):
        self.r_,_ = pearson_correlation(X,y)
        
        self.beta_ = self.r_*stddev(y)/stddev(X)
        self.alpha_ = mean(y) - mean(X)*self.beta_
        return self
        
    def predict(self,X,y=None):
        try:
            getattr(self, "beta_")
            getattr(self, "alpha_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        
        return self.alpha_ + X*self.beta_

In [ ]:
ols = OLS().fit(test_1['x'],test_1['y'])
prediction = ols.predict(test_1['x'])
print(f'slope:{ols.beta_:.4f},intercept:{ols.alpha_:.4f}')

In [ ]:
plt.scatter(test_1['x'],test_1['y'])
plt.plot(test_1['x'],prediction)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
lr = LinearRegression().fit(test_1['x'].reshape(-1,1),test_1['y'].reshape(-1,1))
print(f'slope:{lr.coef_[0][0]:.4f},intercept:{lr.intercept_[0]:.4f}\nMSE: {mean_squared_error(prediction,test_1["x"]):.4f}')